In [582]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, Dataset, TensorDataset
from tqdm import tqdm
import copy
import matplotlib.pyplot as plt
from collections import OrderedDict
import random
import torch.nn.functional as F
import torch.nn.functional as func
import collections
from sklearn.model_selection import train_test_split
from collections import Counter

In [583]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True' # 解决由于多次加载 OpenMP 相关动态库而引起的冲突

In [584]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jan  3 16:48:22 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:21:00.0 Off |                  Off |
| 30%   30C    P8              11W / 350W |    561MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [585]:

class LinearBottleNeck(nn.Module):

    def __init__(self, in_channels, out_channels, stride, t=6, class_num=100):
        super().__init__()

        self.residual = nn.Sequential(
            nn.Conv2d(in_channels, in_channels * t, 1),
            nn.BatchNorm2d(in_channels * t),
            nn.ReLU6(inplace=True),

            nn.Conv2d(in_channels * t, in_channels * t, 3, stride=stride, padding=1, groups=in_channels * t),
            nn.BatchNorm2d(in_channels * t),
            nn.ReLU6(inplace=True),

            nn.Conv2d(in_channels * t, out_channels, 1),
            nn.BatchNorm2d(out_channels)
        )

        self.stride = stride
        self.in_channels = in_channels
        self.out_channels = out_channels

    def forward(self, x):

        residual = self.residual(x)

        if self.stride == 1 and self.in_channels == self.out_channels:
            residual += x

        return residual

# MobileNetV2（比lenet更复杂的CNN网络）网络中的线性瓶颈结构，原文中用于CIFAR-100任务
class MobileNetV2(nn.Module):

    def __init__(self, class_num=20):
        super().__init__()

        self.pre = nn.Sequential(
            nn.Conv2d(3, 32, 1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU6(inplace=True)
        )

        self.stage1 = LinearBottleNeck(32, 16, 1, 1)
        self.stage2 = self._make_stage(2, 16, 24, 2, 6)
        self.stage3 = self._make_stage(3, 24, 32, 2, 6)
        self.stage4 = self._make_stage(4, 32, 64, 2, 6)
        self.stage5 = self._make_stage(3, 64, 96, 1, 6)
        self.stage6 = self._make_stage(3, 96, 160, 1, 6)
        self.stage7 = LinearBottleNeck(160, 320, 1, 6)

        self.conv1 = nn.Sequential(
            nn.Conv2d(320, 1280, 1),
            nn.BatchNorm2d(1280),
            nn.ReLU6(inplace=True)
        )

        self.conv2 = nn.Conv2d(1280, class_num, 1)

    def forward(self, x):
        x = self.pre(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.stage4(x)
        x = self.stage5(x)
        x = self.stage6(x)
        x = self.stage7(x)
        x = self.conv1(x)
        x = F.adaptive_avg_pool2d(x, 1)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)

        return x

    def _make_stage(self, repeat, in_channels, out_channels, stride, t):

        layers = []
        layers.append(LinearBottleNeck(in_channels, out_channels, stride, t))

        while repeat - 1:
            layers.append(LinearBottleNeck(out_channels, out_channels, 1, t))
            repeat -= 1

        return nn.Sequential(*layers)

def mobilenetv2():
    return MobileNetV2()


# FedMut中采用的cnn模型
class CNNCifar(nn.Module):
    def __init__(self):
        super(CNNCifar, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x, start_layer_idx=0, logit=False):
        if start_layer_idx < 0:  #
            return self.mapping(x, start_layer_idx=start_layer_idx, logit=logit)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        result = {'activation' : x}
        x = x.view(-1, 16 * 5 * 5)
        result['hint'] = x
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        result['representation'] = x
        x = self.fc3(x)
        result['output'] = x
        return result

    def mapping(self, z_input, start_layer_idx=-1, logit=True):
        z = z_input
        z = self.fc3(z)

        result = {'output': z}
        if logit:
            result['logit'] = z
        return result
    
def cnncifar():
    return CNNCifar()

In [586]:
# def test_inference(model, test):
#     """ Returns the test accuracy and loss.
#     """
#     tensor_x = torch.Tensor(test[0]).to(device)
#     tensor_y = torch.Tensor(test[1]).to(device)
#     test_dataset = TensorDataset(tensor_x, tensor_y)

#     model.eval()
#     loss, total, correct = 0.0, 0.0, 0.0

#     criterion = nn.CrossEntropyLoss()
#     testloader = DataLoader(test_dataset, batch_size=bc_size,
#                             shuffle=True)

#     for batch_idx, (images, labels) in enumerate(testloader):
#         with torch.no_grad():  # 在测试过程中不需要计算梯度，节省内存和加速计算
#         # Inference
#             outputs = model(images)
#             batch_loss = criterion(outputs, labels.long())
#             loss += batch_loss.item() * labels.size(0) # 计算损失值，更好反映模型输出概率分布与真实标签的差距

#         # Prediction
#             _, pred_labels = torch.max(outputs, 1)
#             pred_labels = pred_labels.view(-1)
#             correct += torch.sum(torch.eq(pred_labels, labels)).item()
#             total += len(labels)
#     #print(correct,"/",total)
#     accuracy = correct/total
    
#     print("Testing accuracy: {:.2f}", accuracy)
#     return accuracy, loss


In [587]:
# 新的测试：针对整个测试数据集的测试
def test_inference(net_glob, dataset_test):
    # testing
    acc_test, loss_test = test_img(net_glob, dataset_test)

    # print("Testing accuracy: {:.2f}".format(acc_test))

    return acc_test.item()

def test_img(net_g, datatest):
    net_g.eval()
    # testing
    # test loss代表在测试集上的平均损失（对测试数据的预测输出与真实标签的差距）
    test_loss = 0
    correct = 0
    data_loader = DataLoader(datatest, batch_size=bc_size)
    l = len(data_loader)
    with torch.no_grad():
        for idx, (data, target) in enumerate(data_loader):
            if gpu != -1:
                data, target = data.cuda(), target.cuda()
            log_probs = net_g(data)['output']
            # sum up batch loss
            test_loss += F.cross_entropy(log_probs, target, reduction='sum').item()
            # get the index of the max log-probability
            y_pred = log_probs.data.max(1, keepdim=True)[1]
            correct += y_pred.eq(target.data.view_as(y_pred)).long().cpu().sum()

    test_loss /= len(data_loader.dataset)
    accuracy = 100.00 * correct / len(data_loader.dataset)
    if verbose:
        print('\nTest set: Average loss: {:.4f} \nAccuracy: {}/{} ({:.2f}%)\n'.format(
            test_loss, correct, len(data_loader.dataset), accuracy))
    return accuracy, test_loss

In [588]:
# 将CIFAR-100的100个类别转为20个类别（粒度更粗，降低任务复杂度）
def sparse2coarse(targets):
    """Convert Pytorch CIFAR100 sparse targets to coarse targets.

    Usage:
        trainset = torchvision.datasets.CIFAR100(path)
        trainset.targets = sparse2coarse(trainset.targets)
    """
    coarse_labels = np.array([ 4,  1, 14,  8,  0,  6,  7,  7, 18,  3,
                               3, 14,  9, 18,  7, 11,  3,  9,  7, 11,
                               6, 11,  5, 10,  7,  6, 13, 15,  3, 15,
                               0, 11,  1, 10, 12, 14, 16,  9, 11,  5,
                               5, 19,  8,  8, 15, 13, 14, 17, 18, 10,
                               16, 4, 17,  4,  2,  0, 17,  4, 18, 17,
                               10, 3,  2, 12, 12, 16, 12,  1,  9, 19,
                               2, 10,  0,  1, 16, 12,  9, 13, 15, 13,
                              16, 19,  2,  4,  6, 19,  5,  5,  8, 19,
                              18,  1,  2, 15,  6,  0, 17,  8, 14, 13])
    return coarse_labels[targets]

In [589]:
# 共有6w个图像，其中5w训练，1w测试
def CIFAR100():
    '''Return Cifar100
    '''
    train_dataset = torchvision.datasets.CIFAR100(root='../data/CIFAR-100',
                                            train=True,
                                            transform=transforms.ToTensor(),
                                            download=True)
    test_dataset = torchvision.datasets.CIFAR100(root='../data/CIFAR-100',
                                            train=False,
                                            transform=transforms.ToTensor(),
                                            download=True)
    total_img,total_label = [],[]
    for imgs,labels in train_dataset:
        total_img.append(imgs.numpy())
        total_label.append(labels)
    # for imgs,labels in test_dataset:
    #     total_img.append(imgs.numpy())
    #     total_label.append(labels) 
    total_img = np.array(total_img)
    total_label = np.array(sparse2coarse(total_label))

    cifar = [total_img, total_label]
    return cifar, test_dataset


In [590]:
# 基于 Dirichlet 分布 来模拟non-IID。返回一个形状为 (client_num, class_num) 的概率矩阵，每一行代表一个客户端对各类别的概率分布。
def get_prob(non_iid, client_num, class_num = 20):
    # Modify：我之后加上的
    if data_random_fix:
        np.random.seed(seed_num)  # 固定种子，确保数据抽样一致
    
    return np.random.dirichlet(np.repeat(non_iid, class_num), client_num)

In [591]:
# 全部用于构建训练集
def create_data_all_train(prob, size_per_client, dataset, N=20):
    total_each_class = size_per_client * np.sum(prob, 0)
    data, label = dataset

    if data_random_fix:
        np.random.seed(seed_num)  # 固定种子，确保数据抽样一致
        random.seed(seed_num)

    # 为每个类别随机采样数据
    all_class_set = []
    for i in range(N):
        size = total_each_class[i]
        sub_data = data[label == i]
        sub_label = label[label == i]

        rand_indx = np.random.choice(len(sub_data), size=int(size), replace=False).astype(int)
        sub2_data, sub2_label = sub_data[rand_indx], sub_label[rand_indx]
        all_class_set.append((sub2_data, sub2_label))

    index = [0] * N
    clients = []

    for m in range(prob.shape[0]):  # 遍历客户端
        labels, images = [], []  # 训练数据

        for n in range(N):
            # 100%用于训练
            start, end = index[n], index[n] + int(prob[m][n] * size_per_client)
            image, label = all_class_set[n][0][start:end], all_class_set[n][1][start:end]

            # 记录当前类别的数据分配进度
            index[n] += int(prob[m][n] * size_per_client)

            labels.extend(label)
            images.extend(image)

        clients.append((np.array(images), np.array(labels)))

    return clients

# 80%构建训练集，20%构建测试集
def create_data(prob, size_per_client, dataset, N=20):
    total_each_class = size_per_client * np.sum(prob, 0)
    data, label = dataset

    # Modify：我之后加上的
    if data_random_fix:
        np.random.seed(seed_num)  # 固定种子，确保数据抽样一致
        random.seed(seed_num)

    # 为每个类别随机采样数据
    all_class_set = []
    for i in range(N):
        size = total_each_class[i]
        sub_data = data[label == i]
        sub_label = label[label == i]

        rand_indx = np.random.choice(len(sub_data), size=int(size), replace=False).astype(int)
        sub2_data, sub2_label = sub_data[rand_indx], sub_label[rand_indx]
        all_class_set.append((sub2_data, sub2_label))

    index = [0] * N
    clients, test = [], []

    for m in range(prob.shape[0]):  # 遍历客户端
        labels, images = [], []  # 训练数据
        tlabels, timages = [], [] # 测试数据

        for n in range(N):
            # 80%用于训练，20%用于测试
            # 这里的int向下取整，会导致实际的数据量比计算略小
            start, end = index[n], index[n] + int(prob[m][n] * size_per_client * 0.8)
            test_start, test_end = end, index[n] + int(prob[m][n] * size_per_client)

            image, label = all_class_set[n][0][start:end], all_class_set[n][1][start:end]
            test_image, test_label = all_class_set[n][0][test_start:test_end], all_class_set[n][1][test_start:test_end]

            # 记录当前类别的数据分配进度
            index[n] += int(prob[m][n] * size_per_client)

            labels.extend(label)
            images.extend(image)

            tlabels.extend(test_label)
            timages.extend(test_image)

        clients.append((np.array(images), np.array(labels)))
        test.append((np.array(timages), np.array(tlabels)))

    return clients, test

In [592]:

# 合并所有客户端的测试数据 （上面讲测试数据分成了不同的客户端）
# 但并没有使用，用途不明
def comb_client_test_func(client_test_data):
    comb_client_test_image = []
    comb_client_test_label = []
    for i in range(client_num):
        comb_client_test_image.extend(list(client_test_data[i][0]))
        comb_client_test_label.extend(list(client_test_data[i][1]))
    
    # 将测试图片和标签合并为 numpy 数组
    comb_client_test_image = np.array(comb_client_test_image)
    comb_client_test_label = np.array(comb_client_test_label)
    
    label_count = Counter(comb_client_test_label)
    print("测试集类别分布：")
    for label, count in sorted(label_count.items()):
        print(f"类别 {label}: {count} 个样本")
    
    return [comb_client_test_image, comb_client_test_label]

In [593]:
# 从数据集中按类别均匀抽取子集，并按照指定的比例 percentage 进行缩减，同时对数据进行随机打乱
def select_subset(whole_set, percentage):
    
    # Modify：我之后加上的
    if data_random_fix:
        np.random.seed(seed_num)  # 固定种子，确保数据抽样一致
        random.seed(seed_num)
    
    a = whole_set[0]
    b = whole_set[1]
    if len(a) != len(b):
        raise ValueError("Both arrays should have the same length.")

    if not 0 <= percentage <= 1:
        raise ValueError("Percentage must be between 0 and 1.")

    unique_classes = np.unique(b)

    a_prime = []
    b_prime = []

    for cls in unique_classes:
        indices = np.where(b == cls)[0]
        subset_size = int(len(indices) * percentage)

        selected_indices = np.random.choice(indices, subset_size, replace=False)

        a_prime.extend(a[selected_indices])
        b_prime.extend(b[selected_indices])

    a_prime, b_prime = np.array(a_prime), np.array(b_prime)

    # Shuffle arrays to randomize the order of elements
    shuffle_indices = np.random.permutation(len(a_prime))
    a_prime, b_prime = a_prime[shuffle_indices], b_prime[shuffle_indices]

    return [a_prime, b_prime]

In [594]:
# 本地训练并更新权重，返回更新后的模型权重、平均训练损失以及第一个迭代的梯度信息
def update_weights(model_weight, dataset, learning_rate, local_epoch):
    model = cnncifar().to(device)
    model.load_state_dict(model_weight)

    model.train()
    epoch_loss = []
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()

    Tensor_set = TensorDataset(torch.Tensor(dataset[0]).to(device), torch.Tensor(dataset[1]).to(device))
    data_loader = DataLoader(Tensor_set, batch_size=bc_size, shuffle=True)

    first_iter_gradient = None  # 初始化变量来保存第一个iter的梯度

    for iter in range(local_epoch):
        batch_loss = []
        for batch_idx, (images, labels) in enumerate(data_loader):
            model.zero_grad()
            outputs = model(images)
            loss = criterion(outputs['output'], labels.long())
            loss.backward()
            optimizer.step()
            batch_loss.append(loss.item()/images.shape[0])

            # 保存第一个iter的梯度
            if iter == 0 and batch_idx == 0:
                first_iter_gradient = {}
                for name, param in model.named_parameters():
                    first_iter_gradient[name] = param.grad.clone()
                # 保存 BatchNorm 层的 running mean 和 running variance
                for name, module in model.named_modules():
                    if isinstance(module, nn.BatchNorm2d):
                        first_iter_gradient[name + '.running_mean'] = module.running_mean.clone()
                        first_iter_gradient[name + '.running_var'] = module.running_var.clone()

        epoch_loss.append(sum(batch_loss)/len(batch_loss))

    return model.state_dict(), sum(epoch_loss) / len(epoch_loss), first_iter_gradient

In [595]:
# 计算模型权重的差异，并根据学习率 lr 对权重差异进行缩放
def weight_differences(n_w, p_w, lr):
    w_diff = copy.deepcopy(n_w)
    for key in w_diff.keys():
        if 'num_batches_tracked' in key:
            continue
        w_diff[key] = (p_w[key] - n_w[key]) * lr
    return w_diff

In [596]:
# 也是本地训练，不过引入了Fed-C的权重修正机制
def update_weights_correction(model_weight, dataset, learning_rate, local_epoch, c_i, c_s):
    model = cnncifar().to(device)
    model.load_state_dict(model_weight)

    model.train()
    epoch_loss = []
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()

    Tensor_set = TensorDataset(torch.Tensor(dataset[0]).to(device), torch.Tensor(dataset[1]).to(device))
    data_loader = DataLoader(Tensor_set, batch_size=bc_size, shuffle=True)

    for iter in range(local_epoch):
        batch_loss = []
        for batch_idx, (images, labels) in enumerate(data_loader):
            model.zero_grad()
            outputs = model(images)
            loss = criterion(outputs['output'], labels.long())
            loss.backward()
            optimizer.step()
            batch_loss.append(loss.sum().item()/images.shape[0])
        epoch_loss.append(sum(batch_loss)/len(batch_loss))
        corrected_graident = weight_differences(c_i, c_s, learning_rate)
        orginal_model_weight = model.state_dict()
        corrected_model_weight = weight_differences(corrected_graident, orginal_model_weight, 1)  # 这里缩放权重为1
        model.load_state_dict(corrected_model_weight)

    return model.state_dict(),  sum(epoch_loss) / len(epoch_loss)

In [597]:
def average_weights(w):
    """
    Returns the average of the weights.
    """
    w_avg = copy.deepcopy(w[0])
    for key in w_avg.keys():
        if 'num_batches_tracked' in key:
            continue
        for i in range(1, len(w)):
            w_avg[key] += w[i][key]
        w_avg[key] = torch.div(w_avg[key], len(w))
    return w_avg

In [598]:
# baseline: server-only
def server_only(initial_w, global_round, gamma, E):
    test_model = cnncifar().to(device)
    train_w = copy.deepcopy(initial_w)
    test_acc = []
    train_loss = []
    
    
    for round in tqdm(range(global_round)):
        # if gamma > 0.001:
        #     gamma = gamma * 0.99
        # Server side local training
        
        
        # 从comb中（论文中说明为全部训练数据）选择固定比率的server数据（并且是保证类别均衡的）
        # server_data = select_subset(comb_client_data, server_percentage)
                
        update_server_w, round_loss, _ = update_weights(train_w, server_data, gamma, E)
        train_w = update_server_w
        test_model.load_state_dict(train_w)
        train_loss.append(round_loss)
        
        # 新的测试（针对全部测试数据进行）
        test_acc.append(test_inference(test_model, test_dataset))
    
        # Test Accuracy
        # test_a = 0
        # for i in client_test_data:
        #     ac = test_inference(test_model,i)[0]
        #     test_a = test_a + ac
        # test_a = test_a/len(client_test_data)
        # test_acc.append(test_a)
        # print(test_a)
    return test_acc, train_loss

In [599]:
def fedavg(initial_w, global_round, eta, K, M):
    test_model = cnncifar().to(device)
    train_w = copy.deepcopy(initial_w)
    test_acc = []
    train_loss = []
    for round in tqdm(range(global_round)):
        local_weights, local_loss = [], []
        # Client side local training
        # if eta > 0.001:
        #     eta = eta * 0.99
        sampled_client = random.sample(range(client_num), M)
        for i in sampled_client:
            update_client_w, client_round_loss, _ = update_weights(train_w, client_data[i], eta, K)
            local_weights.append(update_client_w)
            local_loss.append(client_round_loss)

        train_w = average_weights(local_weights)

        # Test Accuracy
        test_model.load_state_dict(train_w)
        loss_avg = sum(local_loss)/ len(local_loss)
        train_loss.append(loss_avg)
        
        # 新的测试（针对全部测试数据进行）
        test_acc.append(test_inference(test_model, test_dataset))
            
        # test_a = 0
        # for i in client_test_data:
        #     ac = test_inference(test_model,i)[0]
        #     test_a = test_a + ac
        # test_a = test_a/len(client_test_data)
        # test_acc.append(test_a)
#         print(test_a)
    return test_acc, train_loss


In [600]:
def hybridFL(initial_w, global_round, eta, K, M):
    """
    HybridFL算法：FedAvg改进，服务器也作为一个普通客户端参与训练。
    
    参数:
    - initial_w: 初始模型权重
    - global_round: 全局训练轮数
    - eta: 学习率
    - K: 本地训练轮数
    - M: 每轮采样的客户端数量
    """
    test_model = cnncifar().to(device)  # 初始化测试模型
    train_w = copy.deepcopy(initial_w)     # 当前全局权重
    test_acc = []                          # 保存每轮测试精度
    train_loss = []                        # 保存每轮训练损失
    
    for round in tqdm(range(global_round)):
        local_weights, local_loss = [], []  # 存储每个客户端/服务器的权重和损失

        # 随机采样 M 个客户端
        sampled_client = random.sample(range(client_num), M)

        # 客户端本地训练
        for i in sampled_client:
            update_client_w, client_round_loss, _ = update_weights(train_w, client_data[i], eta, K)
            local_weights.append(update_client_w)
            local_loss.append(client_round_loss)

        # 服务器参与训练
        update_server_w, server_round_loss, _ = update_weights(train_w, server_data, eta, K)
        local_weights.append(update_server_w)   # 将服务器权重加入列表
        local_loss.append(server_round_loss)    # 将服务器损失加入列表

        # 权重聚合
        train_w = average_weights(local_weights)

        # 评估模型性能
        test_model.load_state_dict(train_w)
        loss_avg = sum(local_loss) / len(local_loss)
        train_loss.append(loss_avg)

        # 新的测试（针对全部测试数据进行）
        test_acc.append(test_inference(test_model, test_dataset))
    
        
        # test_a = 0
        # for i in client_test_data:  # 遍历所有客户端测试数据
        #     ac = test_inference(test_model, i)[0]
        #     test_a += ac
        # test_a = test_a / len(client_test_data)
        # test_acc.append(test_a)
        
        # # 打印每轮的结果
        # print(f"Round {round + 1}: Test Accuracy = {test_a:.4f}, Train Loss = {loss_avg:.4f}")
    
    return test_acc, train_loss


In [601]:
def CLG_SGD(initial_w, global_round, eta, gamma, K, E, M):
    test_model = cnncifar().to(device)
    train_w = copy.deepcopy(initial_w)
    test_acc = []
    train_loss = []
    
    for round in tqdm(range(global_round)):
        # 学习率衰减，这里默认注释掉了
        # if eta > 0.001:
        #     eta = eta * 0.99
        # if gamma > 0.001:
        #     gamma = gamma * 0.99
        local_weights, local_loss = [], []
        # Client side local training
        # 从总共client_num客户端中选择M个训练
        sampled_client = random.sample(range(client_num), M)
        for i in sampled_client:
            update_client_w, client_round_loss, _ = update_weights(train_w, client_data[i], eta, K)
            local_weights.append(update_client_w)
            local_loss.append(client_round_loss)
        train_w = average_weights(local_weights)
        # Server side local training
        
        
        # 从comb中（论文中说明为全部训练数据）选择固定比率的server数据（并且是保证类别均衡的）
        # TODO_241216:这里是每一轮都重新选择数据（但保证类别比例是一样的，都是按照comb中的比例），我的场景中可以这样吗？
        # server_data = select_subset(comb_client_data, server_percentage)
        
        update_server_w, round_loss, _ = update_weights(train_w, server_data, gamma, E)
        train_w = update_server_w
        local_loss.append(round_loss)

        # Test Accuracy
        test_model.load_state_dict(train_w)
        loss_avg = sum(local_loss)/ len(local_loss)
        train_loss.append(loss_avg)   # 计算所有客户端和服务器一起的平均损失

        # 新的测试（针对全部测试数据进行）
        test_acc.append(test_inference(test_model, test_dataset))
    
        # test_a = 0
        # # 遍历客户端测试数据，计算平均准确率
        # for i in client_test_data:
        #     ac = test_inference(test_model,i)[0]
        #     test_a = test_a + ac
        # test_a = test_a/len(client_test_data)
        # test_acc.append(test_a)
#         print(test_a)
    return test_acc, train_loss

In [602]:

def CLG_Mut(net_glob, global_round, eta, gamma, K, E, M):
    
    net_glob.train()
    
    test_model = cnncifar().to(device)
    train_w = copy.deepcopy(net_glob.state_dict())
    test_acc = []
    train_loss = []
    
    w_locals = []
    for i in range(M):
        w_locals.append(copy.deepcopy(net_glob.state_dict()))
    
    delta_list = []
    max_rank = 0
    w_old = copy.deepcopy(net_glob.state_dict())
    w_old_s1 = copy.deepcopy(net_glob.state_dict())
    
    for round in tqdm(range(global_round)):
        # 学习率衰减，这里默认注释掉了
        # if eta > 0.001:
        #     eta = eta * 0.99
        # if gamma > 0.001:
        #     gamma = gamma * 0.99
        local_weights, local_loss = [], []
        # Client side local training
        # 从总共client_num客户端中选择M个训练
        idxs_users = np.random.choice(range(client_num), M, replace=False)
        for i, idx in enumerate(idxs_users):
            net_glob.load_state_dict(w_locals[i])
            
            update_client_w, client_round_loss, _ = update_weights(copy.deepcopy(net_glob.state_dict()), client_data[idx], eta, K)
            w_locals[i] = copy.deepcopy(update_client_w)
            local_loss.append(client_round_loss)

        # Global Model Generation
        w_agg = Aggregation(w_locals, None)  
        
        # Server side local training
        update_server_w, round_loss, _ = update_weights(w_agg, server_data, gamma, E)
        local_loss.append(round_loss)

        # Test Accuracy
        test_model.load_state_dict(update_server_w)
        loss_avg = sum(local_loss)/ len(local_loss)
        train_loss.append(loss_avg)   # 计算所有客户端和服务器一起的平均损失


        # 新的测试（针对全部测试数据进行）
        test_acc.append(test_inference(test_model, test_dataset))

        # 按照server训练的方向，进行mutation
        w_delta = FedSub(update_server_w, w_agg, 1.0)
        # 计算模型更新w_delta的L2范数（平方和），衡量模型更新程度的大小
        rank = delta_rank(w_delta)
        # print(rank)
        if rank > max_rank:
            max_rank = rank
        alpha = radius  # 论文中的alpha，衡量Mutation的幅度
        # alpha = min(max(args.radius, max_rank/rank),(10.0-args.radius) * (1 - iter/args.epochs) + args.radius)
        w_locals = mutation_spread(
            round, update_server_w, M, w_delta, alpha
        )

    return test_acc, train_loss


def mutation_spread(iter, w_glob, m, w_delta, alpha):

    w_locals_new = []
    ctrl_cmd_list = []
    ctrl_rate = mut_acc_rate * (
        1.0 - min(iter * 1.0 / mut_bound, 1.0)
    )  # 论文中的βt，随着iter逐渐从β0减小到0

    # k代表模型中的参数数量，对每个参数按照client数量分配v（论文中是按照每一层分配）
    for k in w_glob.keys():
        ctrl_list = []
        for i in range(0, int(m / 2)):
            ctrl = random.random()  # 随机数，范围：[0,1)
            # 这里分ctrl感觉没什么必要，shuffle后都会随机掉
            if ctrl > 0.5:
                ctrl_list.append(1.0)
                ctrl_list.append(1.0 * (-1.0 + ctrl_rate))
            else:
                ctrl_list.append(1.0 * (-1.0 + ctrl_rate))
                ctrl_list.append(1.0)
        random.shuffle(ctrl_list)  # 打乱列表
        ctrl_cmd_list.append(ctrl_list)
    cnt = 0
    for j in range(m):
        w_sub = copy.deepcopy(w_glob)
        if not (cnt == m - 1 and m % 2 == 1):
            ind = 0
            for k in w_sub.keys():
                w_sub[k] = w_sub[k] + w_delta[k] * ctrl_cmd_list[ind][j] * alpha
                ind += 1
        cnt += 1
        w_locals_new.append(w_sub)

    return w_locals_new



# 加权平均聚合，lens代表了权重，如果没有定义就是普通平均（FedMut就每定义）
def Aggregation(w, lens):
    w_avg = None
    if lens == None:
        total_count = len(w)
        lens = []
        for i in range(len(w)):
            lens.append(1.0)
    else:
        total_count = sum(lens)

    for i in range(0, len(w)):
        if i == 0:
            w_avg = copy.deepcopy(w[0])
            for k in w_avg.keys():
                w_avg[k] = w[i][k] * lens[i]
        else:
            for k in w_avg.keys():
                w_avg[k] += w[i][k] * lens[i]

    for k in w_avg.keys():
        w_avg[k] = torch.div(w_avg[k], total_count)

    return w_avg



def FedSub(w, w_old, weight):
    w_sub = copy.deepcopy(w)
    for k in w_sub.keys():
        w_sub[k] = (w[k] - w_old[k]) * weight

    return w_sub

def delta_rank(delta_dict):
    cnt = 0
    dict_a = torch.Tensor(0)
    s = 0
    for p in delta_dict.keys():
        a = delta_dict[p]
        a = a.view(-1)
        if cnt == 0:
            dict_a = a
        else:
            dict_a = torch.cat((dict_a, a), dim=0)

        cnt += 1
        # print(sim)
    s = torch.norm(dict_a, dim=0)
    return s



In [603]:
# 设置训练参数 -- CIFAR100

# 随机性
data_random_fix = False  # 是否固定数据采样的随机性
seed_num = 42

gpu = 1  # 默认使用gpu 1 (第二个)
verbose = False  # 调试模式，输出一些中间信息

client_num = 100
non_iid = 1.0  # Dirichlet 分布参数，数值越小数据越不均匀可根据需要调整
size_per_client = 400  # 每个客户端的数据量（训练）
server_percentage = 0.01  # 服务器端用于微调的数据比例


# 模型相关
momentum = 0.9
weight_decay = 0.0001  # 模型权重衰减参数，强制参数向0靠拢（和学习率衰减不一样！）这个是给我的原始代码中就是这样
bc_size = 128
num_classes = 20  # 分别数量，CIFAR100中是20（FedMut和CLGG都是这么采用的）

# 联邦训练的超参数
global_round = 100  # 全局训练轮数，可根据需要调整
eta = 0.1  # 客户端端学习率，从{0.01, 0.1, 1}中调优
gamma = 0.05  # 服务器端学习率 从{0.005， 0.05， 0.5中调有}
K = 5  # 客户端本地训练轮数，从1，3，5中选
E = 5  # 服务器本地训练轮数，从1，3，5中选
M = 10  # 每一轮抽取客户端

# FedMut中参数
radius = 1  # alpha，控制mutation的幅度
mut_acc_rate = 0.3  # 论文中的β0
mut_bound = 50  # Tb

In [604]:
# 准备数据集
# 这部分是我加的

cifar, test_dataset = CIFAR100()
prob = get_prob(non_iid, client_num, class_num=20)
# client_data, client_test_data = create_data(prob, size_per_client, cifar, N=20)
client_data = create_data_all_train(prob, size_per_client, cifar, N=20)   # 这里改为全部构建训练集

# 将测试标签转换为粗类别
test_dataset.targets = sparse2coarse(test_dataset.targets)

# 如果需要确保测试标签为整数类型
test_dataset.targets = test_dataset.targets.astype(int)



all_images = []
all_labels = []
for data in client_data:
    all_images.extend(data[0])
    all_labels.extend(data[1])
comb_client_data = [np.array(all_images), np.array(all_labels)]

# 输出comb_client_data情况
imgs, lbls = comb_client_data
lbls = np.array(lbls)
total_count = len(lbls)
unique_classes, counts = np.unique(lbls, return_counts=True)

# 创建一个长度为20的数组记录各类别计数，默认0
class_counts = [0]*20
for cls, cnt in zip(unique_classes, counts):
    class_counts[cls] = cnt

# 打印格式：Total: 总数 类别0计数 类别1计数 ... 类别19计数
print("Traning Client Total: {}".format(" ".join([str(total_count)] + [str(c) for c in class_counts])))


# 打印每个客户端训练数据情况（只输出前10个）
for i, (imgs, lbls) in enumerate(client_data[:10]):
    lbls = np.array(lbls)
    total_count = len(lbls)
    unique_classes, counts = np.unique(lbls, return_counts=True)
    # 创建一个长度为20的数组记录各类别计数，默认0
    class_counts = [0]*20
    for cls, cnt in zip(unique_classes, counts):
        class_counts[cls] = cnt
    # 打印格式：Client i: 总数 类别0计数 类别1计数 ... 类别19计数
    print("Client {}: {}".format(i, " ".join([str(total_count)] + [str(c) for c in class_counts])))
    

# 提前生成固定的服务器数据
# Modify: 这是我后来修改的
server_data = select_subset(comb_client_data, server_percentage)

s_imgs, s_lbls = server_data
s_lbls = np.array(s_lbls)
total_count = len(s_lbls)
unique_classes, counts = np.unique(s_lbls, return_counts=True)
class_counts = [0]*20
for cls, cnt in zip(unique_classes, counts):
    class_counts[cls] = cnt

# 输出格式: Server: 总数 类别0计数 类别1计数 ... 类别19计数
print("Server: {}".format(" ".join([str(total_count)] + [str(c) for c in class_counts])))
# print("  前5个标签: ", lbls[:5])
# print("  前5个数据形状: ", [server_data[0][j].shape for j in range(min(5, len(server_data[0])))])





# 用了FedMut中定义的CNN网络

init_model = cnncifar().to(device)
initial_w = copy.deepcopy(init_model.state_dict())


# CLG_Mut训练，这里不同在于直接传初始化后的模型
test_acc, train_loss = CLG_Mut(init_model, global_round, eta, gamma, K, E, M)

# 打印训练过程中的结果
print("CLG-Mut 训练完成！")
print("各轮平均测试精度:", test_acc)
print("各轮平均训练损失:", train_loss)
print("最终测试精度:", test_acc[-1] if len(test_acc) > 0 else "无数据")


# Server-only训练
test_acc, train_loss = server_only(initial_w, global_round, gamma, E)

# 打印训练过程中的结果
print("Server only 训练完成！")
print("各轮平均测试精度:", test_acc)
print("各轮平均训练损失:", train_loss)
print("最终测试精度:", test_acc[-1] if len(test_acc) > 0 else "无数据")


# fedavg训练
test_acc, train_loss = fedavg(initial_w, global_round, eta, K, M)

# 打印训练过程中的结果
print("fedavg训练完成！")
print("各轮平均测试精度:", test_acc)
print("各轮平均训练损失:", train_loss)
print("最终测试精度:", test_acc[-1] if len(test_acc) > 0 else "无数据")



# # hybridfl训练
# test_acc, train_loss = hybridFL(initial_w, global_round, eta, K, M)

# # 打印训练过程中的结果
# print("hrbridFL训练完成！")
# print("各轮平均测试精度:", test_acc)
# print("各轮平均训练损失:", train_loss)
# print("最终测试精度:", test_acc[-1] if len(test_acc) > 0 else "无数据")

# CLG_SGD训练
test_acc, train_loss = CLG_SGD(initial_w, global_round, eta, gamma, K, E, M)

# 打印训练过程中的结果
print("CLG_SGD 训练完成！")
print("各轮平均测试精度:", test_acc)
print("各轮平均训练损失:", train_loss)
print("最终测试精度:", test_acc[-1] if len(test_acc) > 0 else "无数据")


Files already downloaded and verified
Files already downloaded and verified
Traning Client Total: 38984 2125 2295 2060 2198 2031 1924 1882 1957 1543 1736 1802 1926 2380 1900 1580 1811 1807 1968 1949 2110
Client 0: 388 10 0 8 27 39 18 5 17 23 38 21 31 34 13 25 12 5 32 27 3
Client 1: 389 36 5 8 14 3 8 32 29 7 23 23 46 12 37 5 4 4 11 17 65
Client 2: 388 7 66 91 15 27 5 8 4 18 2 3 3 25 32 15 15 20 2 17 13
Client 3: 392 3 57 18 13 1 15 4 0 4 6 64 23 4 9 34 12 2 43 52 28
Client 4: 390 9 37 7 16 63 30 36 3 8 9 4 14 13 0 7 2 37 5 12 78
Client 5: 391 12 18 2 24 14 9 13 23 64 21 10 3 57 17 14 11 3 0 10 66
Client 6: 388 11 83 11 9 33 23 22 15 9 0 44 16 31 23 17 16 5 2 8 10
Client 7: 391 34 14 24 32 2 23 5 37 1 27 35 17 1 6 6 47 0 27 32 21
Client 8: 390 7 46 28 1 22 23 6 121 3 47 29 7 16 5 0 9 4 7 5 4
Client 9: 391 1 0 20 2 90 54 13 67 4 44 1 4 10 9 6 12 11 7 6 30
Server: 381 21 22 20 21 20 19 18 19 15 17 18 19 23 19 15 18 18 19 19 21


100%|██████████| 100/100 [01:54<00:00,  1.15s/it]


CLG-Mut 训练完成！
各轮平均测试精度: [5.0, 5.0, 8.170000076293945, 7.860000133514404, 5.25, 5.0, 5.0, 5.0, 5.0, 5.409999847412109, 6.889999866485596, 5.0, 6.0, 5.0, 5.0, 5.019999980926514, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]
各轮平均训练损失: [0.14807310909545335, 0.12808904112926103, 0.14995896833310976, 0.12555251702311168, 0.11756698692880603, 0.12219834175961038, 0.12784419863513666, 0.11690884630686214, 0.12576225893989657, 0.13625988774359613, 0.12149970174878834, 0.12668237003079613, 0.1215928284632695, 0.13018853477045333, 0.13574195371515307, 0.13853057234005042, 0.13997352341851504, 0.1264681775710638, 0.128924654968663

100%|██████████| 100/100 [01:03<00:00,  1.57it/s]


Server only 训练完成！
各轮平均测试精度: [5.0, 5.769999980926514, 5.420000076293945, 7.059999942779541, 9.319999694824219, 6.130000114440918, 9.550000190734863, 9.550000190734863, 11.140000343322754, 11.350000381469727, 9.0, 11.5, 13.359999656677246, 13.140000343322754, 13.239999771118164, 15.069999694824219, 12.699999809265137, 15.100000381469727, 14.050000190734863, 14.270000457763672, 15.220000267028809, 14.270000457763672, 13.34000015258789, 13.5600004196167, 13.1899995803833, 14.75, 14.380000114440918, 13.489999771118164, 14.100000381469727, 14.1899995803833, 14.260000228881836, 14.270000457763672, 14.210000038146973, 14.239999771118164, 14.180000305175781, 14.220000267028809, 14.1899995803833, 14.180000305175781, 14.149999618530273, 14.210000038146973, 14.239999771118164, 14.25, 14.199999809265137, 14.229999542236328, 14.180000305175781, 14.210000038146973, 14.199999809265137, 14.1899995803833, 14.260000228881836, 14.210000038146973, 14.180000305175781, 14.199999809265137, 14.210000038146973,

100%|██████████| 100/100 [01:48<00:00,  1.09s/it]


fedavg训练完成！
各轮平均测试精度: [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.690000057220459, 5.0, 5.440000057220459, 5.0, 7.78000020980835, 5.0, 5.0, 5.0, 4.989999771118164, 3.6700000762939453, 5.0, 5.28000020980835, 5.019999980926514, 5.0, 5.0, 5.690000057220459, 6.28000020980835, 5.730000019073486, 7.099999904632568, 7.230000019073486, 7.730000019073486, 9.109999656677246, 6.480000019073486, 7.159999847412109, 9.369999885559082, 8.779999732971191, 7.440000057220459, 11.050000190734863, 9.130000114440918, 11.529999732971191, 8.800000190734863, 14.979999542236328, 7.019999980926514, 11.640000343322754, 11.529999732971191, 14.029999732971191, 13.369999885559082, 10.149999618530273, 13.130000114440918, 12.949999809265137, 7.880000114440918, 11.920000076293945, 13.0600004196167, 12.850000381469727, 15.260000228881836, 13.8100004196167, 14.880000114440918, 14.550000190734863, 13.0, 14.0, 16.350000381469727, 16.09000015258789, 14.220000267028809, 14.569999694824219, 14.670000076293945, 14.539999961853027, 17.07

100%|██████████| 100/100 [01:53<00:00,  1.13s/it]

CLG_SGD 训练完成！
各轮平均测试精度: [5.0, 5.0, 5.0, 5.0, 5.059999942779541, 6.429999828338623, 7.059999942779541, 9.260000228881836, 8.09000015258789, 9.319999694824219, 8.739999771118164, 9.449999809265137, 9.880000114440918, 10.3100004196167, 10.5, 9.289999961853027, 11.5, 11.630000114440918, 10.539999961853027, 12.8100004196167, 13.470000267028809, 12.880000114440918, 14.670000076293945, 14.25, 15.710000038146973, 15.239999771118164, 16.479999542236328, 16.540000915527344, 16.260000228881836, 14.109999656677246, 16.40999984741211, 14.989999771118164, 15.8100004196167, 15.960000038146973, 16.65999984741211, 15.979999542236328, 16.420000076293945, 16.6200008392334, 15.6899995803833, 16.6200008392334, 17.18000030517578, 17.18000030517578, 16.959999084472656, 16.31999969482422, 16.6200008392334, 16.06999969482422, 16.520000457763672, 16.6200008392334, 15.880000114440918, 16.850000381469727, 16.90999984741211, 16.479999542236328, 16.260000228881836, 16.670000076293945, 16.43000030517578, 16.67000007